In [ ]:
from numpy.random import seed
seed(101)

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

tf.random.set_seed(101)

import os

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
import shutil
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# dataset_dir = '../../Documents/skin-cancer-mnist-ham10000/'
# base_dir = 'base_dir'
# train_dir = os.path.join(base_dir, 'train_dir')
# val_dir = os.path.join(base_dir, 'val_dir')

# df_data = pd.read_csv(dataset_dir + '/HAM10000_metadata.csv')

# # this will tell us how many images are associated with each lesion_id
# df = df_data.groupby('lesion_id').count()

# # now we filter out lesion_id's that have only one image associated with it
# df = df[df['image_id'] == 1]
# df.reset_index(inplace=True)


# # here we identify lesion_id's that have duplicate images and those that have only
# # one image.

# def identify_duplicates(x):
#     unique_list = list(df['lesion_id'])
    
#     if x in unique_list:
#         return 'no_duplicates'
#     else:
#         return 'has_duplicates'
    
# # create a new colum that is a copy of the lesion_id column
# df_data['duplicates'] = df_data['lesion_id']
# # apply the function to this new column
# df_data['duplicates'] = df_data['duplicates'].apply(identify_duplicates)

# # now we filter out images that don't have duplicates
# df = df_data[df_data['duplicates'] == 'no_duplicates']

# # now we create a val set using df because we are sure that none of these images
# # have augmented duplicates in the train set
# y = df['dx']

# _, df_val = train_test_split(df, test_size=0.17, random_state=101, stratify=y)

# # This set will be df_data excluding all rows that are in the val set

# # This function identifies if an image is part of the train
# # or val set.
# def identify_val_rows(x):
#     # create a list of all the lesion_id's in the val set
#     val_list = list(df_val['image_id'])
    
#     if str(x) in val_list:
#         return 'val'
#     else:
#         return 'train'

# # identify train and val rows

# # create a new colum that is a copy of the image_id column
# df_data['train_or_val'] = df_data['image_id']
# # apply the function to this new column
# df_data['train_or_val'] = df_data['train_or_val'].apply(identify_val_rows)
   
# # filter out train rows
# df_train = df_data[df_data['train_or_val'] == 'train']


# print(len(df_train))
# print(len(df_val))

In [ ]:
# for v2
dataset_dir = '../../Documents/skin-cancer-mnist-ham10000/'
base_dir = 'base_dir_v2'
train_dir = os.path.join(base_dir, 'train_dir')
val_dir = os.path.join(base_dir, 'val_dir')

df_data = pd.read_csv(dataset_dir + '/HAM10000_metadata.csv')


#X = df_data.drop('dx', axis=1)
y = df_data['dx']

df_train, df_val = train_test_split(df_data, test_size=0.1, random_state=101, stratify=y)

print(df_train.shape)
print(df_val.shape)

In [ ]:
train_path = base_dir + '/train_dir'
valid_path = base_dir + '/val_dir'

num_train_samples = len(df_train)
num_val_samples = len(df_val)
train_batch_size = 10
val_batch_size = 10
image_size = 224

train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)


datagen = ImageDataGenerator(
    preprocessing_function= \
    tf.keras.applications.mobilenet.preprocess_input)

train_batches = datagen.flow_from_directory(train_path,
                                            target_size=(image_size,image_size),
                                            batch_size=train_batch_size)

valid_batches = datagen.flow_from_directory(valid_path,
                                            target_size=(image_size,image_size),
                                            batch_size=val_batch_size)

# Note: shuffle=False causes the test dataset to not be shuffled
test_batches = datagen.flow_from_directory(valid_path,
                                            target_size=(image_size,image_size),
                                            batch_size=1,
                                            shuffle=False)

from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy

def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

def top_2_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=2)

class_weights = {
    0: 1.0, # akiec
    1: 1.0, # bcc
    2: 1.0, # bkl
    3: 1.0, # df
    4: 3.0, # mel # Try to make the model more sensitive to Melanoma.
    5: 1.0, # nv
    6: 1.0, # vasc
}

In [ ]:
# create a copy of a mobilenet model
mobile = tf.keras.applications.mobilenet.MobileNet()

mobile.summary()
print(type(mobile.layers))
# How many layers does MobileNet have?
print(len(mobile.layers))

# CREATE THE MODEL ARCHITECTURE

# Exclude the last 5 layers of the above model.
# This will include all layers up to and including global_average_pooling2d_1
x = mobile.layers[-6].output

# Create a new dense layer for predictions
# 7 corresponds to the number of classes
x = Dropout(0.25)(x)
predictions = Dense(7, activation='softmax')(x)

# inputs=mobile.input selects the input layer, outputs=predictions refers to the
# dense layer we created above.

model = Model(inputs=mobile.input, outputs=predictions)

# We need to choose how many layers we actually want to be trained.

# Here we are freezing the weights of all layers except the
# last 23 layers in the new model.
# The last 23 layers of the model will be trained.

for layer in model.layers[:-23]:
    layer.trainable = False

In [ ]:
model.compile(Adam(lr=0.01), loss='categorical_crossentropy', 
              metrics=[categorical_accuracy, top_2_accuracy, top_3_accuracy])

filepath = "mobilenet_model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_categorical_accuracy', verbose=1, 
                             save_best_only=True, mode='max')

reduce_lr = ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=2, 
                                   verbose=1, mode='max', min_lr=0.00001)
                              
                              
callbacks_list = [checkpoint, reduce_lr]

history = model.fit_generator(train_batches, steps_per_epoch=train_steps, 
                              class_weight=class_weights,
                    validation_data=valid_batches,
                    validation_steps=val_steps,
                    epochs=30, verbose=1,
                   callbacks=callbacks_list)


In [ ]:
# Here the the last epoch will be used.

val_loss, val_cat_acc, val_top_2_acc, val_top_3_acc = \
model.evaluate_generator(test_batches, 
                        steps=len(df_val))

print('val_loss:', val_loss)
print('val_cat_acc:', val_cat_acc)
print('val_top_2_acc:', val_top_2_acc)
print('val_top_3_acc:', val_top_3_acc)

In [ ]:
# Here the best epoch will be used.

model.load_weights('mobilenet_model.h5')

val_loss, val_cat_acc, val_top_2_acc, val_top_3_acc = \
model.evaluate_generator(test_batches, 
                        steps=len(df_val))

print('val_loss:', val_loss)
print('val_cat_acc:', val_cat_acc)
print('val_top_2_acc:', val_top_2_acc)
print('val_top_3_acc:', val_top_3_acc)

In [ ]:
# display the loss and accuracy curves
acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
train_top2_acc = history.history['top_2_accuracy']
val_top2_acc = history.history['val_top_2_accuracy']
train_top3_acc = history.history['top_3_accuracy']
val_top3_acc = history.history['val_top_3_accuracy']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.figure()

plt.plot(epochs, acc, 'bo', label='Training cat acc')
plt.plot(epochs, val_acc, 'b', label='Validation cat acc')
plt.title('Training and validation cat accuracy')
plt.legend()
plt.figure()


plt.plot(epochs, train_top2_acc, 'bo', label='Training top2 acc')
plt.plot(epochs, val_top2_acc, 'b', label='Validation top2 acc')
plt.title('Training and validation top2 accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, train_top3_acc, 'bo', label='Training top3 acc')
plt.plot(epochs, val_top3_acc, 'b', label='Validation top3 acc')
plt.title('Training and validation top3 accuracy')
plt.legend()


plt.show()

In [ ]:
def train_mobilenet_v2(last_layers_training=-1):
    IMG_SHAPE = (224, 224, 3)

    # Create the base model from the pre-trained model MobileNet V2
    mobilev2 = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                                   weights='imagenet')
#     mobilev2.summary()
#     print(len(mobilev2.layers))

    x = mobilev2.layers[-2].output
    predictions = Dense(7, activation='softmax')(x)
    model = Model(inputs=mobilev2.input, outputs=predictions)

    for layer in model.layers[:last_layers_training]:
        layer.trainable = False
        
    base_learning_rate = 0.0001

    model.compile(Adam(lr=0.01), loss='categorical_crossentropy', 
                  metrics=[categorical_accuracy, top_2_accuracy, top_3_accuracy])

    filepath = "mobilenetv2_model_" + str(last_layers_training) + ".h5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_categorical_accuracy', verbose=1, 
                                 save_best_only=True, mode='max')

    reduce_lr = ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=2, 
                                       verbose=1, mode='max', min_lr=0.00001)


    callbacks_list = [checkpoint, reduce_lr]

    history = model.fit_generator(train_batches, steps_per_epoch=train_steps, 
                                  class_weight=class_weights,
                        validation_data=valid_batches,
                        validation_steps=val_steps,
                        epochs=20, verbose=1,
                       callbacks=callbacks_list)
    
    # Here the best epoch will be used.
    model.load_weights(filepath)

    val_loss, val_cat_acc, val_top_2_acc, val_top_3_acc = \
    model.evaluate_generator(test_batches, 
                            steps=len(df_val))

    print('val_loss:', val_loss)
    print('val_cat_acc:', val_cat_acc)
    print('val_top_2_acc:', val_top_2_acc)
    print('val_top_3_acc:', val_top_3_acc)

    acc = history.history['categorical_accuracy']
    val_acc = history.history['val_categorical_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    train_top2_acc = history.history['top_2_accuracy']
    val_top2_acc = history.history['val_top_2_accuracy']
    train_top3_acc = history.history['top_3_accuracy']
    val_top3_acc = history.history['val_top_3_accuracy']
    epochs = range(1, len(acc) + 1)

    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    plt.figure()

    plt.plot(epochs, acc, 'bo', label='Training cat acc')
    plt.plot(epochs, val_acc, 'b', label='Validation cat acc')
    plt.title('Training and validation cat accuracy')
    plt.legend()
    plt.figure()


    plt.plot(epochs, train_top2_acc, 'bo', label='Training top2 acc')
    plt.plot(epochs, val_top2_acc, 'b', label='Validation top2 acc')
    plt.title('Training and validation top2 accuracy')
    plt.legend()
    plt.figure()
    plt.plot(epochs, train_top3_acc, 'bo', label='Training top3 acc')
    plt.plot(epochs, val_top3_acc, 'b', label='Validation top3 acc')
    plt.title('Training and validation top3 accuracy')
    plt.legend()

    plt.show()

In [ ]:
# train_mobilenet_v2(last_layers_training=-5) # lock until last 5 layers
# train_mobilenet_v2(last_layers_training=-13) # lock until last 5 layers + block_16

# for i in range(1, 10): # lock until last 5 layers + block_16 + block_15 ~ block_6  
#     train_mobilenet_v2(last_layers_training=-13-9*i)

# for i in range(10, 16):
#     train_mobilenet_v2(last_layers_training=-13-9*i)
    
# train_mobilenet_v2(last_layers_training=-157)

In [ ]:
from MobileNet_V3 import build_mobilenet_v3
model = build_mobilenet_v3(input_size=224, num_classes=7, model_type='small', pooling_type='avg', include_top=True)

print(model.summary())
print(len(model.layers))

In [ ]:
base_learning_rate = 0.0001

model.compile(Adam(lr=0.1), loss='categorical_crossentropy', 
              metrics=[categorical_accuracy, top_2_accuracy, top_3_accuracy])

filepath = "mobilenetv3_model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_categorical_accuracy', verbose=1, 
                             save_best_only=True, mode='max')

reduce_lr = ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=2, 
                                   verbose=1, mode='max', min_lr=0.00001)

callbacks_list = [checkpoint]
history = model.fit_generator(train_batches, steps_per_epoch=train_steps, 
                              class_weight=class_weights,
                    validation_data=valid_batches,
                    validation_steps=val_steps,
                    epochs=80, verbose=1,
                   callbacks=callbacks_list)

In [ ]:
# Here the best epoch will be used.

model.load_weights(filepath)

val_loss, val_cat_acc, val_top_2_acc, val_top_3_acc = \
model.evaluate_generator(test_batches, 
                        steps=len(df_val))

print('val_loss:', val_loss)
print('val_cat_acc:', val_cat_acc)
print('val_top_2_acc:', val_top_2_acc)
print('val_top_3_acc:', val_top_3_acc)

acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
train_top2_acc = history.history['top_2_accuracy']
val_top2_acc = history.history['val_top_2_accuracy']
train_top3_acc = history.history['top_3_accuracy']
val_top3_acc = history.history['val_top_3_accuracy']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.figure()

plt.plot(epochs, acc, 'bo', label='Training cat acc')
plt.plot(epochs, val_acc, 'b', label='Validation cat acc')
plt.title('Training and validation cat accuracy')
plt.legend()
plt.figure()


plt.plot(epochs, train_top2_acc, 'bo', label='Training top2 acc')
plt.plot(epochs, val_top2_acc, 'b', label='Validation top2 acc')
plt.title('Training and validation top2 accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, train_top3_acc, 'bo', label='Training top3 acc')
plt.plot(epochs, val_top3_acc, 'b', label='Validation top3 acc')
plt.title('Training and validation top3 accuracy')
plt.legend()


plt.show()